In [1]:
%use adventOfCode

In [2]:
val aoc = AocClient.fromEnv().interactiveDay(2025, 10)
aoc.viewPartOne()

--- Day 10: Factory --- 
 Just across the hall, you find a large factory. Fortunately, the Elves here have plenty of time to decorate. Unfortunately, it's because the factory machines are all offline, and none of the Elves can figure out the initialization procedure. 
 The Elves do have the manual for the machines, but the section detailing the initialization procedure was eaten by a Shiba Inu . All that remains of the manual are some indicator light diagrams, button wiring schematics, and joltage requirements for each machine. 
 For example: 
 [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
 
 The manual describes one machine per line. Each line contains a single indicator light diagram in [ square brackets ] , one or more button wiring schematics in ( parentheses ) , and joltage requirements in { curly braces } . 
 To start a machine, its indicator lights must match those shown in the diagram, where . means off and # means on . The machine has the number of indicator lights shown, but its indicator lights are all initially off . 
 So, an indicator light diagram like [.##.] means that the machine has four indicator lights which are initially off and that the goal is to simultaneously configure the first light to be off, the second light to be on, the third to be on, and the fourth to be off. 
 You can toggle the state of indicator lights by pushing any of the listed buttons . Each button lists which indicator lights it toggles, where 0 means the first light, 1 means the second light, and so on. When you push a button, each listed indicator light either turns on (if it was off) or turns off (if it was on). You have to push each button an integer number of times; there's no such thing as " 0.5 presses " (nor can you push a button a negative number of times). 
 So, a button wiring schematic like (0,3,4) means that each time you push that button, the first, fourth, and fifth indicator lights would all toggle between on and off. If the indicator lights were [#.....] , pushing the button would change them to be [...##.] instead. 
 Because none of the machines are running, the joltage requirements are irrelevant and can be safely ignored. 
 You can push each button as many times as you like. However, to save on time, you will need to determine the fewest total presses required to correctly configure all indicator lights for all machines in your list. 
 There are a few ways to correctly configure the first machine: 
 [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7} 
 
 You could press the first three buttons once each, a total of 3 button presses. 
 You could press (1,3) once, (2,3) once, and (0,1) twice, a total of 4 button presses. 
 You could press all of the buttons except (1,3) once each, a total of 5 button presses. 
 
 However, the fewest button presses required is 2 . One way to do this is by pressing the last two buttons ( (0,2) and (0,1) ) once each. 
 The second machine can be configured with as few as 3 button presses: 
 [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2} 
 One way to achieve this is by pressing the last three buttons ( (0,4) , (0,1,2) , and (1,2,3,4) ) once each. 
 The third machine has a total of six indicator lights that need to be configured correctly: 
 [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5} 
 The fewest presses required to correctly configure it is 2 ; one way to do this is by pressing buttons (0,3,4) and (0,1,2,4,5) once each. 
 So, the fewest button presses required to correctly configure the indicator lights on all of the machines is 2 + 3 + 2 = 7 . 
 Analyze each machine's indicator light diagram and button wiring schematics. What is the fewest button presses required to correctly configure the indicator lights on all of the machines?

In [3]:
val exampleInput = """
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
""".trim()

In [4]:
data class Indicators(val list: List<Boolean>) {
    override fun toString(): String = "[${list.joinToString("") { if (it) "#" else "." }}]"

    fun press(button: Button) =
        Indicators(
            list.mapIndexed { i: Int, state: Boolean ->
                if (i in button.set) !state else state
            }
        )
}

data class Button(val set: Set<Int>) {
    override fun toString(): String = set.joinToString(",", "(", ")")
}

data class JoltageRequirements(val list: List<Int>) {
    override fun toString(): String = list.joinToString(",", "{", "}")

    // pt2
    fun press(button: Button) = JoltageRequirements(
        list.mapIndexed { i, it ->
            if (i in button.set) it + 1 else it
        }
    )
}

class Machine(
    val indicatorTarget: Indicators,
    val buttons: List<Button>,
    val joltageRequirements: JoltageRequirements,
) {
    val emptyIndicators = Indicators(List(indicatorTarget.list.size) { false })
    val emptyJoltageRequirements = JoltageRequirements(List(joltageRequirements.list.size) { 0 })

    override fun toString(): String = "$indicatorTarget $buttons $joltageRequirements"
}


fun String.parse(): List<Machine> = trim().lines().map {
    val input = it.split(" ")
    val indicatorTarget = input.first()
        .removeSurrounding("[", "]")
        .map { it == '#' }
    val buttons = input.drop(1).dropLast(1).map {
        Button(
            it.removeSurrounding("(", ")")
                .split(",")
                .map { it.toInt() }
                .toSet())
    }
    val joltageRequirements = input.last()
        .removeSurrounding("{", "}")
        .split(",")
        .map { it.toInt() }

    Machine(
        indicatorTarget = Indicators(indicatorTarget),
        buttons = buttons,
        joltageRequirements = JoltageRequirements(joltageRequirements),
    )
}

In [5]:
val exampleMachine = exampleInput.parse().first()
exampleMachine

[.##.] [(3), (1,3), (2), (2,3), (0,2), (0,1)] {3,5,4,7}

In [6]:
val machine = exampleMachine

val queue = ArrayDeque<Indicators>().also { it.add(machine.emptyIndicators) }
val visitedStatesWithPathLength = mutableMapOf<Indicators, Int>(machine.emptyIndicators to 0)

while (queue.isNotEmpty()) {
    val currentState: Indicators = queue.removeFirst()
    val pathLength = visitedStatesWithPathLength[currentState]!!

    for (button in machine.buttons) {
        val afterPressed = currentState.press(button)
        if (visitedStatesWithPathLength[afterPressed]?.let { it < pathLength + 1 } == true) continue
        queue += afterPressed
        visitedStatesWithPathLength[afterPressed] = pathLength + 1
    }
}

visitedStatesWithPathLength[machine.indicatorTarget]!!

2

In [7]:
aoc.input().parse().joinToString("\n")

[##.#..] [(3,4,5), (1,3,5), (4,5), (2), (0,2), (0,1,4), (0,2,3,4), (0,3)] {38,17,32,60,37,41}
[#.###] [(1,2,4), (1,2), (0,2,3,4)] {13,19,32,13,27}
[#####] [(1,2,3,4), (0,1,3), (1,3), (2,3,4), (2,3), (0,1,3,4)] {16,32,35,59,42}
[.####..#] [(2,7), (0,4,7), (0,3,4,5,7), (1,5), (0,1,2,5,6,7), (0,1,2,4), (1,3,4,5)] {39,39,29,35,49,54,7,55}
[.#.#..#.#] [(0,6,8), (0,2,3,7), (0,1,6,7,8), (2,4,5,8), (2,3,4,5,7,8), (0,1,2,3,4,5,8), (1,2,3,6,8)] {27,5,31,12,24,24,18,7,42}
[###.] [(1,2), (0,2,3), (0,3), (0,2), (1,3), (2)] {16,14,18,25}
[.#..#] [(3,4), (0,2), (1,3), (0,1,2), (0,1,4)] {14,12,9,2,7}
[#.##.] [(0,4), (1,3), (1,4), (0,3), (0,2,3)] {16,12,10,14,14}
[.#.#####] [(2,6), (1,3,4,6), (6,7), (1,3,4,7), (0,1,2), (0,1,3,5), (0,2,3,5,6), (0,2,3), (2,3,7)] {37,52,34,56,30,25,56,26}
[#...##] [(2,3,5), (0,4,5), (0,1,4,5), (0,1,4), (0,4), (0,3), (1,3,4)] {47,27,1,3,49,29}
[...#.#.##.] [(1,2,3,4,5,6,7,9), (0,2,6), (0,3,6), (1,3,4,5,8,9), (0,1,4,5,7,8,9), (4,7,8), (1,2,5,8), (3,5,7,8)] {40,38,38,18,45,4

In [8]:
fun Machine.smallestNumberOfPresses(): Int {
    val queue = ArrayDeque<Indicators>().also { it.add(emptyIndicators) }
    val visitedStatesWithPathLength = mutableMapOf<Indicators, Int>(emptyIndicators to 0)

    while (queue.isNotEmpty()) {
        val currentState: Indicators = queue.removeFirst()
        val pathLength = visitedStatesWithPathLength[currentState]!!

        for (button in buttons) {
            val afterPressed = currentState.press(button)
            if (visitedStatesWithPathLength[afterPressed]?.let { it < pathLength + 1 } == true) continue
            if (afterPressed == indicatorTarget) return pathLength + 1

            visitedStatesWithPathLength[afterPressed] = pathLength + 1
            queue += afterPressed
        }
    }
    error("No solution found")
}


In [9]:
exampleInput.parse().sumOf { it.smallestNumberOfPresses() }

7

In [10]:
val input = aoc.input().parse()

input.withIndex().sumOf { (i, it) ->
    it.smallestNumberOfPresses()
}

396

In [11]:
//aoc.submitPartOne(396)

In [12]:
aoc.viewPartTwo()

--- Part Two --- 
 All of the machines are starting to come online! Now, it's time to worry about the joltage requirements. 
 Each machine needs to be configured to exactly the specified joltage levels to function properly. Below the buttons on each machine is a big lever that you can use to switch the buttons from configuring the indicator lights to increasing the joltage levels. (Ignore the indicator light diagrams.) 
 The machines each have a set of numeric counters tracking its joltage levels, one counter per joltage requirement. The counters are all initially set to zero . 
 So, joltage requirements like {3,5,4,7} mean that the machine has four counters which are initially 0 and that the goal is to simultaneously configure the first counter to be 3 , the second counter to be 5 , the third to be 4 , and the fourth to be 7 . 
 The button wiring schematics are still relevant: in this new joltage configuration mode, each button now indicates which counters it affects, where 0 means the first counter, 1 means the second counter, and so on. When you push a button, each listed counter is increased by 1 . 
 So, a button wiring schematic like (1,3) means that each time you push that button, the second and fourth counters would each increase by 1 . If the current joltage levels were {0,1,2,3} , pushing the button would change them to be {0,2,2,4} . 
 You can push each button as many times as you like. However, your finger is getting sore from all the button pushing, and so you will need to determine the fewest total presses required to correctly configure each machine's joltage level counters to match the specified joltage requirements. 
 Consider again the example from before: 
 [.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
 
 Configuring the first machine's counters requires a minimum of 10 button presses. One way to do this is by pressing (3) once, (1,3) three times, (2,3) three times, (0,2) once, and (0,1) twice. 
 Configuring the second machine's counters requires a minimum of 12 button presses. One way to do this is by pressing (0,2,3,4) twice, (2,3) five times, and (0,1,2) five times. 
 Configuring the third machine's counters requires a minimum of 11 button presses. One way to do this is by pressing (0,1,2,3,4) five times, (0,1,2,4,5) five times, and (1,2) once. 
 So, the fewest button presses required to correctly configure the joltage level counters on all of the machines is 10 + 12 + 11 = 33 . 
 Analyze each machine's joltage requirements and button wiring schematics. What is the fewest button presses required to correctly configure the joltage level counters on all of the machines?

In [13]:
fun Machine.smallestNumberOfPressesPt2(): Int {
    val queue = ArrayDeque<JoltageRequirements>().also { it.add(emptyJoltageRequirements) }
    val visitedStatesWithPathLength = mutableMapOf<JoltageRequirements, Int>(emptyJoltageRequirements to 0)

    while (queue.isNotEmpty()) {
        val currentState: JoltageRequirements = queue.removeFirst()
        val pathLength = visitedStatesWithPathLength[currentState]!!

        for (button in buttons) {
            val afterPressed = currentState.press(button)
            if (visitedStatesWithPathLength[afterPressed]?.let { it < pathLength + 1 } == true) continue
            if (afterPressed == joltageRequirements) return pathLength + 1

            if ((afterPressed.list zip joltageRequirements.list).any { (after, target) -> after > target }) continue

            visitedStatesWithPathLength[afterPressed] = pathLength + 1
            queue += afterPressed
        }
    }
    error("No solution found")
}

In [17]:
exampleInput.parse().first()

[.##.] [(3), (1,3), (2), (2,3), (0,2), (0,1)] {3,5,4,7}

In [14]:
exampleInput.parse().sumOf { it.smallestNumberOfPressesPt2() }

33

In [19]:
val machine = exampleInput.parse().first()

val maxPresses: Map<Button, Int> = machine.buttons.associateWith { button ->
    button.set.minOf {
        machine.joltageRequirements.list[it]
    }
}

maxPresses

{(3)=7, (1,3)=5, (2)=4, (2,3)=4, (0,2)=3, (0,1)=3}

In [24]:
machine.joltageRequirements.list.withIndex().map { (i, joltage) ->
    joltage to machine.buttons.filter { i in it.set }
}

[(3, [(0,2), (0,1)]), (5, [(1,3), (0,1)]), (4, [(2), (2,3), (0,2)]), (7, [(3), (1,3), (2,3)])]

maybe library necessary...

In [25]:
return
val input: List<Machine> = aoc.input().parse()

aoc.input().parse().sumOf {
    DISPLAY("${it.joltageRequirements}")
    it.smallestNumberOfPressesPt2()
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[25], line 1, column 1: 'return' is not allowed here